In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from tb_incubator.constants import set_project_base_path, QUANTILES, image_path
from tb_incubator.calibrate import plot_output_ranges, add_line_breaks
from tb_incubator.plotting import get_combined_plot
from tb_incubator.input import load_targets
from estival.sampling import tools as esamp

project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]

In [ ]:
file_suffixes = ["xpert_utilisation_improvement_p17_96", "no_xpert_utilisation_improvement_p17_96"]
assumption_titles = {
    "xpert_utilisation_improvement_p17_96": "With Xpert Improvement",
    "no_xpert_utilisation_improvement_p17_96": "Without Xpert Improvement"
}
    
titles = [assumption_titles[suffix] for suffix in file_suffixes]

spaghettis = {}
quantile_outputs = {}

for suffix in file_suffixes:
    spaghetti = pd.read_hdf(calib_out / f'results_{suffix}.hdf', 'spaghetti')
    spaghettis[suffix] = spaghetti
    quantile_outputs[suffix] = esamp.quantiles_for_results(spaghetti, QUANTILES)

targets = load_targets()
out_req = ["notification_log", "adults_prevalence_pulmonary_log"]

In [ ]:
notif_plots = {}
for suffix in file_suffixes:
    notif_plots[suffix] = plot_output_ranges(
        quantile_outputs[suffix], targets, ['notification'], 1, 2010, 2025, 2013, show_legend=True
    )

In [ ]:
adults_prev_plots = {}
for suffix in file_suffixes:
    adults_prev_plots[suffix] = plot_output_ranges(
        quantile_outputs[suffix], targets, ['adults_prevalence_pulmonary'], 1, 2010, 2025, 2013, show_legend=True
    )

In [ ]:
notif_combined = get_combined_plot(
    plot_list=list(notif_plots.values()),
    n_cols=2,
    subplot_titles=titles,
    shared_yaxes=True,
    shared_xaxes=False
)

notif_combined.update_yaxes(
    title=dict(
        text=add_line_breaks(f"<b>Case notifications<b>", max_chars=30),
        font=dict(size=12)  
    ), 
    row=1, col=1
)

#notif_combined.write_image(image_path / 'notifications_comp.png', scale=3)

In [ ]:
adults_prev_combined = get_combined_plot(
    plot_list=list(adults_prev_plots.values()),
    n_cols=2,
    subplot_titles=titles,
    shared_yaxes=True,
    shared_xaxes=False
)

adults_prev_combined.update_yaxes(
    title=dict(
        text=add_line_breaks(f"<b>Prevalence of pulmonary TB among adults (per 100,000/year)<b>", max_chars=30),
        font=dict(size=12) 
    ), 
    row=1, col=1
)

#adults_prev_combined.write_image(image_path / 'adults_prev_comp.png', scale=3)